In [138]:
from functools import wraps
from redis import Redis
from typing import get_type_hints, Dict, Any
import uuid
import time
from pydantic import BaseModel
from redis_om import JsonModel
from dataclasses import dataclass
from collections import defaultdict

In [45]:
class Watch:
    def __init__(self, val:JsonModel, redis_args:Dict=None):
        if not redis_args:
            redis_args = {}
        self.pk = uuid.uuid4().hex
        self.stream_key = f"event:{self.__class__.__name__}:{self.pk}"
        self.redis_conn = Redis(**redis_args)
        self.val = val
    
    def __lshift__(self, val:Any):
        self.redis_conn.xadd("event")
        self.val = val
        print(f"new : {self.val}")
    
    def notify(self, old, new):
        self.redis_conn.xadd(name=self.stream_key,fields={"time":time.time()})

    def __repr__(self) -> str:
        return repr(self.val)
    
    def __str__(self) -> str:
        return str(self.val)



In [156]:
class Anima:
    def __init__(self, *args, **kwargs):
        self.redis_conn = Redis(*args, **kwargs)
        self.watch_index = {}
    
    def watch(outer_self, wrapped:type):
        outer_self.watch_index[hash(wrapped)] = {"class":wrapped,"instances":[]}

        orig_init = wrapped.__init__
        def new_init(self, *args, **kwargs):
            outer_self.watch_index[hash(self.__class__)]["instances"].append(self)
            print(outer_self.watch_index)
            orig_init(self, *args, **kwargs)
        wrapped.__init__ = new_init

        orig_setattr = wrapped.__setattr__
        def new_setattr(self, __name:str, __value:Any):
            outer_self.redis_conn.xadd("lmao", {"lol":1})
            orig_setattr(self, __name, __value)
        wrapped.__setattr__ = new_setattr
        return wrapped

In [157]:
app = Anima()
app.redis_conn.ping()

True

In [158]:
@app.watch
class T:
    ...

In [159]:
t = T()

{3183219: {'class': <class '__main__.T'>, 'instances': [<__main__.T object at 0x7f77c7f086d0>]}}


In [160]:
t.a = "A"

In [161]:
t.a

'A'